In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

from analysis_variables import *

import myPIDselector

import pandas as pd
import seaborn as sns

import ROOT

In [ ]:
h = ROOT.TH1F()

In [ ]:

c1 = ROOT.TCanvas()
c1.cd()
h.Draw()

c1.Update()

#c1.Show()

# rf202

In [ ]:
# Declare observable x
x = ROOT.RooRealVar("x", "x", 0, 10)

# Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and
# their parameters
mean = ROOT.RooRealVar("mean", "mean of gaussians", 5)
sigma1 = ROOT.RooRealVar("sigma1", "width of gaussians", 0.5)
sigma2 = ROOT.RooRealVar("sigma2", "width of gaussians", 1)

sig1 = ROOT.RooGaussian("sig1", "Signal component 1", x, mean, sigma1)
sig2 = ROOT.RooGaussian("sig2", "Signal component 2", x, mean, sigma2)

# Build Chebychev polynomial p.d.f.
a0 = ROOT.RooRealVar("a0", "a0", 0.5, 0., 1.)
a1 = ROOT.RooRealVar("a1", "a1", -0.2, 0., 1.)
bkg = ROOT.RooChebychev("bkg", "Background", x, ROOT.RooArgList(a0, a1))

# Sum the signal components into a composite signal p.d.f.
sig1frac = ROOT.RooRealVar(
    "sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.)
sig = ROOT.RooAddPdf(
    "sig", "Signal", ROOT.RooArgList(sig1, sig2), ROOT.RooArgList(sig1frac))



In [ ]:
# Method 1 - Construct extended composite model
# -------------------------------------------------------------------

# Sum the composite signal and background into an extended pdf
# nsig*sig+nbkg*bkg
nsig = ROOT.RooRealVar("nsig", "number of signal events", 500, 0., 10000)
nbkg = ROOT.RooRealVar(
    "nbkg", "number of background events", 500, 0, 10000)
model = ROOT.RooAddPdf(
    "model",
    "(g1+g2)+a",
    ROOT.RooArgList(
        bkg,
        sig),
    ROOT.RooArgList(
        nbkg,
        nsig))

# Sample, fit and plot extended model
# ---------------------------------------------------------------------

# Generate a data sample of expected number events in x from model
# = model.expectedEvents() = nsig+nbkg
data = model.generate(ROOT.RooArgSet(x))

# Fit model to data, ML term automatically included
model.fitTo(data)

# Plot data and PDF overlaid, expected number of events for p.d.f projection normalization
# rather than observed number of events (==data.numEntries())
xframe = x.frame(ROOT.RooFit.Title("extended ML fit example"))
data.plotOn(xframe)
model.plotOn(xframe, ROOT.RooFit.Normalization(
    1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background component of model with a dashed line
ras_bkg = ROOT.RooArgSet(bkg)
model.plotOn(
    xframe, ROOT.RooFit.Components(ras_bkg), ROOT.RooFit.LineStyle(
        ROOT.kDashed), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))

# Overlay the background+sig2 components of model with a dotted line
ras_bkg_sig2 = ROOT.RooArgSet(bkg, sig2)
model.plotOn(
    xframe, ROOT.RooFit.Components(ras_bkg_sig2), ROOT.RooFit.LineStyle(
        ROOT.kDotted), ROOT.RooFit.Normalization(
            1.0, ROOT.RooAbsReal.RelativeExpected))


In [ ]:
# Print structure of composite p.d.f.
model.Print("t")


# Method 2 - Construct extended components first
# ---------------------------------------------------------------------

# Associated nsig/nbkg as expected number of events with sig/bkg
esig = ROOT.RooExtendPdf("esig", "extended signal p.d.f", sig, nsig)
ebkg = ROOT.RooExtendPdf("ebkg", "extended background p.d.f", bkg, nbkg)

# Sum extended components without coefs
# -------------------------------------------------------------------------

# Construct sum of two extended p.d.f. (no coefficients required)
model2 = ROOT.RooAddPdf("model2", "(g1+g2)+a", ROOT.RooArgList(ebkg, esig))

# Draw the frame on the canvas
c = ROOT.TCanvas("rf202_extendedmlfit", "rf202_extendedmlfit", 600, 600)
ROOT.gPad.SetLeftMargin(0.15)
xframe.GetYaxis().SetTitleOffset(1.4)
xframe.Draw()

c.Update()

c.SaveAs("rf202_extendedmlfit.png")


# From ChatGPT

In [ ]:
df = pd.read_parquet('mes_deltae_background_AFTER_FINAL_CUTS.parquet')
df

In [ ]:
################################################################################
# Argus background PDF
################################################################################
def argus_in_x(x,tag='default'):
    argpar = ROOT.RooRealVar("argpar_"+tag,"Argus shape par "+tag,-20.0,-1000,10000)
    cutoff = ROOT.RooRealVar("cutoff_"+tag,"Argus cutoff "+tag,5.29,-1000,1000)

    argus = ROOT.RooArgusBG("argus_"+tag,"Argus PDF "+tag,x,cutoff,argpar)

    pars = [argpar, cutoff]
    return pars, argus

################################################################################
# Crystal Barrel function: (mES)
################################################################################
def crystal_barrel_x(x,tag="default"):
    meanCB = ROOT.RooRealVar("meanCB","Gaussian #mu (CB) m_{ES}", 5.279, 5.27, 5.29)
    sigmaCB = ROOT.RooRealVar("sigmaCB_"+tag," Gaussian #sigma (CB) m_{ES} "+tag, 0.0028, 0.002, 0.004)
    alphaCB = ROOT.RooRealVar("alphaCB_"+tag, "#alpha (CB) m_{ES} "+tag, 2.0, 1.5, 3.0)
    nCB = ROOT.RooRealVar("nCB","n of CB", 1.0, 0.0, 3.0)

    cb =     ROOT.RooCBShape("CB", "Crystal Barrel Shape PDF "+tag, x, meanCB, sigmaCB, alphaCB, nCB)

    pars = [meanCB,  sigmaCB, alphaCB, nCB]

    return pars, cb



In [ ]:
#!/usr/bin/env python
#import ROOT

# -----------------------------------------------------------
# 1. Define the variables for x and y.
# -----------------------------------------------------------
x = ROOT.RooRealVar("x", "x", 5.2, 5.3)
y = ROOT.RooRealVar("y", "y", -0.2, 0.2)

# -----------------------------------------------------------
# 2. Build the individual PDFs:
#    Crystal Ball for x:
# -----------------------------------------------------------
mean = ROOT.RooRealVar("mean", "Mean of CB", 0, -10, 10)
sigma = ROOT.RooRealVar("sigma", "Width of CB", 1, 0.1, 5)
alpha = ROOT.RooRealVar("alpha", "Alpha of CB", 1.5, 0.1, 5)
n = ROOT.RooRealVar("n", "n of CB", 5, 1, 10)
cb = ROOT.RooCrystalBall("cb", "Crystal Ball PDF", x, mean, sigma, alpha, n)

# -----------------------------------------------------------
# 3. Argus PDF for y:
#    Here y0 is fixed at 10 (the upper bound of y) and chi is negative.
# -----------------------------------------------------------
chi = ROOT.RooRealVar("chi", "Chi of Argus", -20, -100, 0)
y0 = ROOT.RooRealVar("y0", "Threshold", 10)  # fixed threshold
argus = ROOT.RooArgusBG("argus", "Argus PDF", y, y0, chi)

# -----------------------------------------------------------
# 4. Combine the two PDFs (assuming they factorize):
# -----------------------------------------------------------
model = ROOT.RooProdPdf("model", "Crystal Ball x Argus", ROOT.RooArgList(cb, argus))

# -----------------------------------------------------------
# 5. Generate 500 toy events from the combined PDF:
# -----------------------------------------------------------
toyData = model.generate(ROOT.RooArgSet(x, y), 500)

# -----------------------------------------------------------
# 6. Fit the toy data with the model:
# -----------------------------------------------------------
model.fitTo(toyData)

# -----------------------------------------------------------
# 7. Plot the 1-D projections for x and y from the toy data:
# -----------------------------------------------------------
# Projection on x
c1 = ROOT.TCanvas("c1", "Projection on x", 800, 600)
frameX = x.frame()
toyData.plotOn(frameX)
model.plotOn(frameX)
frameX.Draw()
c1.SaveAs("x_projection.png")

c1.Update()

# Projection on y
c2 = ROOT.TCanvas("c2", "Projection on y", 800, 600)
frameY = y.frame()
toyData.plotOn(frameY)
model.plotOn(frameY)
frameY.Draw()
c2.SaveAs("y_projection.png")

c2.Update()

'''
# -----------------------------------------------------------
# 8. Read in the real data from a parquet file.
#    Assumes a file "data.parquet" with columns "x" and "y".
#    ROOT's RDataFrame supports parquet files (ROOT >= 6.22).
# -----------------------------------------------------------
df = pd.read_parquet("mes_deltae_background_AFTER_FINAL_CUTS.parquet")
#tree = df.GetPtr()

x_data = df['mes'].values
y_data = df['DeltaE'].values
data = np.column_stack((x_data, y_data))


# Build a RooDataSet from the TTree.
dataReal = ROOT.RooDataSet("dataReal", "Real Data", ROOT.RooArgSet(x, y), data)

# Fit the real data with the same model.
model.fitTo(dataReal)

# -----------------------------------------------------------
# 9. Plot the projections for the real data:
# -----------------------------------------------------------
# Projection on x
c3 = ROOT.TCanvas("c3", "Real Data Projection on x", 800, 600)
frameX2 = x.frame()
dataReal.plotOn(frameX2)
model.plotOn(frameX2)
frameX2.Draw()
c3.SaveAs("x_projection_real.png")

# Projection on y
c4 = ROOT.TCanvas("c4", "Real Data Projection on y", 800, 600)
frameY2 = y.frame()
dataReal.plotOn(frameY2)
model.plotOn(frameY2)
frameY2.Draw()
c4.SaveAs("y_projection_real.png")

# -----------------------------------------------------------
# 10. Optionally, save the model and data in a RooWorkspace for later use.
# -----------------------------------------------------------
w = ROOT.RooWorkspace("w", "workspace")
# In PyROOT, since 'import' is a reserved keyword, we use getattr to call the method.
getattr(w, "import")(model)
getattr(w, "import")(dataReal)
w.writeToFile("workspace.root")
'''